# 19기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


유저의 인적사항을 입력하면 어떤 제품을 샀는지 그 제품의 카테고리 1을 target으로 하는 knn 진행

데이터 shape 파악


In [2]:
df.shape

(4998, 12)

데이터 통계량 파악

In [3]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,4.998000e+03,4998.000000,4998.000000,4998.000000,3465.000000,1544.000000,4998.000000
mean,1.003015e+06,8.036815,0.421369,5.320128,9.773737,12.766839,9234.655462
std,1.709910e+03,6.442697,0.493828,3.912281,5.101867,4.023591,4999.225081
min,1.000003e+06,0.000000,0.000000,1.000000,2.000000,3.000000,13.000000
25%,1.001560e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5497.750000
50%,1.003040e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8049.000000
75%,1.004417e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12039.000000
max,1.006037e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23913.000000


결측치 파악

In [4]:
df.isnull().sum().to_frame('nan_count')

,nan_count
User_ID,0
Product_ID,0
Gender,0
Age,0
Occupation,0
City_Category,0
Stay_In_Current_City_Years,0
Marital_Status,0
Product_Category_1,0
Product_Category_2,1533


결측지 비율 확인

In [5]:
pd.DataFrame(data=df.isnull().sum()/len(df),columns = ['nan_ratio'])

,nan_ratio
User_ID,0.000000
Product_ID,0.000000
Gender,0.000000
Age,0.000000
Occupation,0.000000
City_Category,0.000000
Stay_In_Current_City_Years,0.000000
Marital_Status,0.000000
Product_Category_1,0.000000
Product_Category_2,0.306723


결측치 처리(product2,product3 = 제거)

In [6]:
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780
...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,M,55+,7,C,1,1,1,16.0,NaN,11701
4994,1000293,P00358242,M,55+,1,C,1,1,8,NaN,NaN,2045
4995,1001908,P00173342,M,55+,13,C,3,1,1,6.0,14.0,4505
4996,1005517,P00336742,M,55+,0,C,2,1,12,NaN,NaN,1413


In [7]:
df = df.drop('Product_Category_2', axis=1)
df = df.drop('Product_Category_3', axis=1)

In [8]:
df = df.drop(df[(df['Product_Category_1'] == 9) | (df['Product_Category_1'] == 17)].index) ## 9번,17번의갯수가 엄청 적으므로 삭제

필요없는 columns 삭제

In [9]:
df = df.drop('User_ID', axis=1)
df = df.drop('Product_ID', axis=1)

In [10]:
df

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Purchase
0,F,0-17,10,A,3,0,5,2010
1,F,0-17,10,A,1,0,7,4483
2,F,0-17,10,A,1,0,1,7696
3,F,0-17,10,A,3,0,2,16429
4,F,0-17,10,A,3,0,3,5780
...,...,...,...,...,...,...,...,...
4993,M,55+,7,C,1,1,1,11701
4994,M,55+,1,C,1,1,8,2045
4995,M,55+,13,C,3,1,1,4505
4996,M,55+,0,C,2,1,12,1413


범위로 되어있는 칼럼들은 범위제거후 대표값으로 설정

In [11]:
df['Age'].unique()

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [12]:
df['Stay_In_Current_City_Years'].unique()

array(['3', '1', '2', '4+', '0'], dtype=object)

In [13]:
def age_groups(values):
    if values == '0-17':
        return 10
    elif values == '18-25':
        return 20
    elif values == '26-35':
        return 30
    elif values =='36-45':
        return 40
    elif values == '46-50':
        return 48
    elif values == '51-55':
        return 53
    else:
        return 55
def CurrentYear(values):
    if values == '0':
        return 0
    elif values == '1':
        return 1
    elif values == '2':
        return 2
    elif values == '3':
        return 3
    else:
        return 4

In [14]:
df['Age'] = df['Age'].apply(age_groups)
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].apply(CurrentYear)

In [15]:
df['Stay_In_Current_City_Years'].unique()

array([3, 1, 2, 4, 0], dtype=int64)

In [16]:
df['Age'].unique()

array([10, 20, 30, 40, 48, 53, 55], dtype=int64)

데이터 타입 확인

In [17]:
df.dtypes

Gender                        object
Age                            int64
Occupation                     int64
City_Category                 object
Stay_In_Current_City_Years     int64
Marital_Status                 int64
Product_Category_1             int64
Purchase                       int64
dtype: object

연속형 변수와 범수형 변수를 파악하여 알맞게 데이터 타입 변환

In [18]:
df['Occupation'] =df['Occupation'].astype(str)
df['Product_Category_1'] =df['Product_Category_1'].astype(str)
df['Marital_Status'] =df['Marital_Status'].astype(str)



In [19]:
df.dtypes

Gender                        object
Age                            int64
Occupation                    object
City_Category                 object
Stay_In_Current_City_Years     int64
Marital_Status                object
Product_Category_1            object
Purchase                       int64
dtype: object

타겟 변수 분리 및 train test set 분리

In [20]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.7, random_state=2023)

In [21]:
train_target = train_df['Product_Category_1']
test_target = test_df['Product_Category_1']
train_data = train_df.drop('Product_Category_1', axis=1)
test_data = test_df.drop('Product_Category_1', axis=1)

In [22]:
train_data

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase
1360,F,30,2,B,3,0,5996
362,M,20,7,A,0,0,8906
1798,M,30,2,A,2,1,11607
3277,F,40,0,C,2,0,8785
4654,M,53,0,B,2,1,1754
...,...,...,...,...,...,...,...
471,M,20,4,A,1,0,7590
4153,F,48,1,C,4,1,8866
885,M,20,19,C,1,0,8119
2747,M,30,7,C,4,0,1517


원핫인코딩

In [23]:
train_data_dum = pd.get_dummies(train_data)
test_data_dum = pd.get_dummies(test_data)



In [24]:
train_data_dum

,Age,Stay_In_Current_City_Years,Purchase,Gender_F,Gender_M,Occupation_0,Occupation_1,Occupation_10,Occupation_11,Occupation_12,...,Occupation_5,Occupation_6,Occupation_7,Occupation_8,Occupation_9,City_Category_A,City_Category_B,City_Category_C,Marital_Status_0,Marital_Status_1
1360,30,3,5996,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
362,20,0,8906,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
1798,30,2,11607,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3277,40,2,8785,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4654,53,2,1754,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,20,1,7590,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4153,48,4,8866,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
885,20,1,8119,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2747,30,4,1517,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0


In [25]:
train_target_dum = pd.get_dummies(train_target)
test_target_dum = pd.get_dummies(test_target)

knn시작

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

In [27]:
params_1 = {
    "n_neighbors": [i for i in range(1, 20, 2)],
    "p": [1, 2], #1은 맨하탄, 2는 유클리드
    "weights": ['uniform']
}

In [28]:
grid_cv = GridSearchCV(knn, param_grid = params_1, cv = 3)

In [29]:
grid_cv.fit(train_data_dum, train_target_dum)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'p': [1, 2], 'weights': ['uniform']})

In [30]:
grid_cv.best_params_

{'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}

In [31]:
knn_1 = KNeighborsClassifier(n_neighbors = 7, p = 2, weights = 'uniform')
knn_1.fit(train_data_dum, train_target_dum)

KNeighborsClassifier(n_neighbors=7)

In [32]:
test_pred = knn_1.predict(test_data_dum)

In [33]:
test_target_dum.shape

(1498, 18)

In [34]:
test_pred.shape

(1498, 18)

In [35]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print("test_data accuracy : ", accuracy_score(test_target_dum, test_pred))
### target변수의 종류가 많다보니 정확도가 좋지는 않은 것 같다

test_data accuracy :  0.8251001335113485
